In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torchvision
import torchvision.datasets
import torchvision.transforms 
from datetime import datetime
from model import Siamese_Network
import os, sys
from torch.utils.data import DataLoader, Dataset

import torch 
import torch.nn as nn
import torch.nn.functional as F


import matplotlib.pyplot as plt
import numpy as np
import random

from PIL import Image
import PIL.ImageOps    
from torch.utils.data import Dataset

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/danielsung/miniconda3/envs/SNNLocalEnv/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/danielsung/miniconda3/envs/SNNLocalEnv/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <CC4BC91F-8B6A-3F9A-B9EB-A2B9D578E202> /Users/danielsung/miniconda3/envs/SNNLocalEnv/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <F2FE5CF8-5B5B-3FAD-ADF8-C77D90F49FC9> /Users/danielsung/miniconda3/envs/SNNLocalEnv/lib/python3.9/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
def imshow(img, text=None):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
        
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

# Plotting data
def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

In [59]:
class SiameseNetworkDataset(Dataset):
    def __init__(self,imageFolderDataset, imageFolderAltered, transform=None):
        self.imageFolderDataset = imageFolderDataset    
        self.imageFolderAltered = imageFolderAltered
        self.transform = transform
        
    def __getitem__(self,index):
        try: 
            img0_tuple = random.choice(self.imageFolderDataset.imgs)

            #We need to approximately 50% of images to be in the same class
            should_get_same_class = random.randint(0,1) 
            # print("\n\n")
            if should_get_same_class:
                while True:
                    #Look untill the same class image is found
                    img1_tuple = random.choice(self.imageFolderAltered.imgs) 
                    if img0_tuple[1] == img1_tuple[1]:
                        # print("img0 and img1", img0_tuple[1], img1_tuple[1])
                        # print("SAMEECLASS:" , img0_tuple, img1_tuple)
                        break
            else:

                while True:
                    #Look untill a different class image is found
                    img1_tuple = random.choice(self.imageFolderAltered.imgs) 
                    if img0_tuple[1] != img1_tuple[1]:
                        # print("img0 and img1", img0_tuple[1], img1_tuple[1])
                        # print("DIFF CLASS:", img0_tuple, img1_tuple)
                        break

            img0 = Image.open(img0_tuple[0])
            img1 = Image.open(img1_tuple[0])

            img0 = img0.convert("L")
            img1 = img1.convert("L")

            if self.transform is not None:
                img0 = self.transform(img0)
                img1 = self.transform(img1)
            
            return img0, img1, torch.from_numpy(np.array([int(img1_tuple[1] != img0_tuple[1])], dtype=np.float32))
        except FileNotFoundError:
            return self.__getitem__(random.randint(0, len(self) - 1))  # Recursively call __getitem__ with a random index

    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [5]:
class Siamese_Network(nn.Module):
    def __init__(self):
        super(Siamese_Network, self).__init__()

        # convolutional neural network
        self.cnn1 = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=11, stride=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(256, 384, kernel_size=3, stride=1),
            nn.ReLU(inplace=True)
        )

        # fully connected layers
        self.fc1 = nn.Sequential(
            nn.Linear(384, 1024),
            nn.ReLU(inplace=True),

            nn.Linear(1024, 256), 
            nn.ReLU(inplace=True),

            nn.Linear(256, 2)
        )
    
    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output
    
    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

In [6]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_dist = F.pairwise_distance(output1, output2, keepdim=True)

        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_dist, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_dist, min=0.0), 2))
        return loss_contrastive

In [21]:

def train(model, device, train_loader, optimizer, epoch, criterion):
    counter = []
    loss_history = []
    iteration_number = 0

    for i, (img0, img1, label) in enumerate(train_loader, 0):

        # Send the images and labels to CUDA
        # img0, img1, label = img0.cuda(), img1.cuda(), label.cuda()

        # Zero the gradients
        optimizer.zero_grad()

        # Pass in the two images into the network and obtain two outputs
        output1, output2 = model(img0, img1)

        # Pass the outputs of the networks and label into the loss function
        loss_contrastive = criterion(output1, output2, label)

        # Calculate the backpropagation
        loss_contrastive.backward()

        # Optimize
        optimizer.step()

        # Every 10 batches print out the loss
        if i % 10 == 0 :
            print(f"Epoch number {epoch}\n Current loss {loss_contrastive.item()}\n")
            iteration_number += 10
            counter.append(iteration_number)
            print(iteration_number)
            loss_history.append(loss_contrastive.item())
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
				epoch, i * len(img0), len(train_loader.dataset),
				100. * i / len(train_loader), loss_contrastive.item()))
        # if iteration_number==:
        #     show_plot(counter, loss_history)



In [8]:
from datetime import datetime
def makedir(path):
	if not os.path.exists(path):
		os.makedirs(path)

# logs the model you used into a text file. 
# More than welcome to modify this function to keep track of the results
def log():
    with open('all_levels_newDataloader.ipynb', 'r') as inputfile:
        textstr = inputfile.read()
        fn = datetime.now().strftime('%Y_%m_%d_%H_%M_%S') + ".txt"
        log_path = "./logs/"
        makedir(log_path)
        with open(log_path + fn, 'w') as outputfile:
            outputfile.write(textstr)
        return fn

In [68]:
# def main():
fn = log()


epochs = 20
directory_train_images = datasets.ImageFolder(root='../../train_images')
directory_train_altered_images = datasets.ImageFolder(root='../../train_altered-easy')

transform_train = transforms.Compose([transforms.Resize((100,100)),
                                    transforms.ToTensor()
                                ])

print("BMP Train set: ", directory_train_images)
trainset = SiameseNetworkDataset(directory_train_images, directory_train_altered_images, transform_train) 

print("LEN: ", len(trainset))
train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

device = torch.device("cpu")
    
print(device)
model = Siamese_Network().to(device)
    
criterion = ContrastiveLoss()

optimizer = optim.Adam(model.parameters(), lr = 0.001)

# Create a simple dataloader just for simple visualization
# vis_dataloader = DataLoader(trainset,
#                         shuffle=True,
#                         num_workers=2,
#                         batch_size=8)

# # Extract one batch
# example_batch = next(iter(vis_dataloader))

# # Example batch is a list containing 2x8 images, indexes 0 and 1, an also the label
# # If the label is 1, it means that it is not the same person, label is 0, same person in both images
# concatenated = torch.cat((example_batch[0], example_batch[1]),0)

# imshow(torchvision.utils.make_grid(concatenated))
# print(example_batch[2].numpy().reshape(-1))

#######################################################

for epoch in range(1, epochs+1):
    print(epoch)
    train(model, device, train_loader, optimizer, epoch, criterion)
    if epoch % 5==0 and epoch!=0:
    # state_dict save file name : siamese_network_train%_test%_lr.pt
        fileName = str(f'Siamese_network_Date:{fn}__Epoch:{epoch}.pt')
        torch.save(model.state_dict(), fileName)

# Locate the test dataset and load it into the SiameseNetworkDataset

# # if __name__ == "__main__":
# directory_test_images = datasets.ImageFolder(root='data/dataset_MS2/test_images')
# # directory_test_altered_images = datasets.ImageFolder(root='data/dataset_MS2/test_altered-hard')
# directory_test_altered_images = datasets.ImageFolder(root='../../test_altered-easy')
# #     main()

BMP Train set:  Dataset ImageFolder
    Number of datapoints: 4200
    Root location: ../../train_images
LEN:  4200
cpu
1
Epoch number 1
 Current loss 2.1177563667297363

10
Train Epoch: 1 [0/4200 (0%)]	Loss: 2.117756
Epoch number 1
 Current loss 1.1080327033996582

20
Train Epoch: 1 [320/4200 (8%)]	Loss: 1.108033
Epoch number 1
 Current loss 0.7482057809829712

30
Train Epoch: 1 [640/4200 (15%)]	Loss: 0.748206
Epoch number 1
 Current loss 0.8134394884109497

40
Train Epoch: 1 [960/4200 (23%)]	Loss: 0.813439
Epoch number 1
 Current loss 0.6556077599525452

50
Train Epoch: 1 [1280/4200 (30%)]	Loss: 0.655608
Epoch number 1
 Current loss 1.071379542350769

60
Train Epoch: 1 [1600/4200 (38%)]	Loss: 1.071380
Epoch number 1
 Current loss 0.6076765656471252

70
Train Epoch: 1 [1920/4200 (45%)]	Loss: 0.607677
Epoch number 1
 Current loss 0.9219051599502563

80
Train Epoch: 1 [2240/4200 (53%)]	Loss: 0.921905
Epoch number 1
 Current loss 0.7724573612213135

90
Train Epoch: 1 [2560/4200 (61%)]	Lo

In [72]:
### TRAIN MEDIUM (TRANSFER LEARNING)

fn = log()
epochs = 20

directory_train_images = datasets.ImageFolder(root='../../train_images_medium')
directory_train_altered_images = datasets.ImageFolder(root='../../train_altered-medium')

transform_train = transforms.Compose([transforms.Resize((100,100)),
                                    transforms.ToTensor()
                                ])

print("BMP Train set: ", directory_train_images)
trainset = SiameseNetworkDataset(directory_train_images, directory_train_altered_images, transform_train) 

print("LEN: ", len(trainset))
train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

device = torch.device("cpu")
    
print(device)
model = Siamese_Network()
model.load_state_dict(torch.load('Siamese_network_Date:2023_08_02_03_45_33.txt__Epoch:20.pt'))
    
criterion = ContrastiveLoss()

optimizer = optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(1, epochs+1):
    print(epoch)
    train(model, device, train_loader, optimizer, epoch, criterion)
    
    if epoch % 5==0 and epoch!=0:
    # state_dict save file name : siamese_network_train%_test%_lr.pt
        fileName = str(f'Siamese_network_Date:{fn}__Epoch:{epoch}__MEDIUM+EASY.pt')
        torch.save(model.state_dict(), fileName)

BMP Train set:  Dataset ImageFolder
    Number of datapoints: 4199
    Root location: ../../train_images_medium
LEN:  4199
cpu
1
Epoch number 1
 Current loss 0.1809084415435791

10
Train Epoch: 1 [0/4199 (0%)]	Loss: 0.180908
Epoch number 1
 Current loss 0.2761816680431366

20
Train Epoch: 1 [320/4199 (8%)]	Loss: 0.276182
Epoch number 1
 Current loss 0.3357505798339844

30
Train Epoch: 1 [640/4199 (15%)]	Loss: 0.335751
Epoch number 1
 Current loss 0.4097706079483032

40
Train Epoch: 1 [960/4199 (23%)]	Loss: 0.409771
Epoch number 1
 Current loss 0.33077746629714966

50
Train Epoch: 1 [1280/4199 (30%)]	Loss: 0.330777
Epoch number 1
 Current loss 0.12564632296562195

60
Train Epoch: 1 [1600/4199 (38%)]	Loss: 0.125646
Epoch number 1
 Current loss 0.11102290451526642

70
Train Epoch: 1 [1920/4199 (45%)]	Loss: 0.111023
Epoch number 1
 Current loss 0.26072534918785095

80
Train Epoch: 1 [2240/4199 (53%)]	Loss: 0.260725
Epoch number 1
 Current loss 0.1609615981578827

90
Train Epoch: 1 [2560/41

In [103]:
### TRAIN HARD (TRANSFER LEARNING)

fn = log()
epochs = 100

directory_train_images = datasets.ImageFolder(root='../../train_images_hard')
directory_train_altered_images = datasets.ImageFolder(root='../../train_altered-hard')

transform_train = transforms.Compose([transforms.Resize((100,100)),
                                    transforms.ToTensor()
                                ])

print("BMP Train set: ", directory_train_images)
trainset = SiameseNetworkDataset(directory_train_images, directory_train_altered_images, transform_train) 

print("LEN: ", len(trainset))
train_loader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True)

device = torch.device("cpu")
    
print(device)
model = Siamese_Network()
model.load_state_dict(torch.load('Siamese_network_Date:2023_08_02_13_07_31.txt__Epoch:20__MEDIUM+EASY.pt'))
    
criterion = ContrastiveLoss()

### 202_M_RIGHT_middle finger

optimizer = optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(1, epochs+1):
    print(epoch)
    train(model, device, train_loader, optimizer, epoch, criterion)
    
    if epoch % 20==0 and epoch!=0:
    # state_dict save file name : siamese_network_train%_test%_lr.pt
        fileName = str(f'Siamese_network_Date:{fn}__Epoch:{epoch}__HARD+MEDIUM+EASY.pt')
        torch.save(model.state_dict(), fileName)

BMP Train set:  Dataset ImageFolder
    Number of datapoints: 4184
    Root location: ../../train_images_hard
LEN:  4184
cpu
1
Epoch number 1
 Current loss 0.06534244120121002

10
Train Epoch: 1 [0/4184 (0%)]	Loss: 0.065342
Epoch number 1
 Current loss 0.02350427582859993

20
Train Epoch: 1 [160/4184 (4%)]	Loss: 0.023504
Epoch number 1
 Current loss 0.030169909819960594

30
Train Epoch: 1 [320/4184 (8%)]	Loss: 0.030170
Epoch number 1
 Current loss 0.16874706745147705

40
Train Epoch: 1 [480/4184 (11%)]	Loss: 0.168747
Epoch number 1
 Current loss 0.029293790459632874

50
Train Epoch: 1 [640/4184 (15%)]	Loss: 0.029294
Epoch number 1
 Current loss 0.1315317153930664

60
Train Epoch: 1 [800/4184 (19%)]	Loss: 0.131532
Epoch number 1
 Current loss 0.06988532096147537

70
Train Epoch: 1 [960/4184 (23%)]	Loss: 0.069885
Epoch number 1
 Current loss 0.2724473476409912

80
Train Epoch: 1 [1120/4184 (27%)]	Loss: 0.272447
Epoch number 1
 Current loss 0.034205373376607895

90
Train Epoch: 1 [1280/41

In [ ]:
def init_random_seed(manual_seed):
    """Init random seed."""
    seed = None
    if manual_seed is None:
        seed = random.randint(1, 10000)
    else:
        seed = manual_seed
    print("use random seed: {}".format(seed))
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    return seed

In [109]:

directory_test_images = datasets.ImageFolder(root='../../test_images_hard')
# directory_test_altered_images = datasets.ImageFolder(root='data/dataset_MS2/test_altered-hard')
directory_test_altered_images = datasets.ImageFolder(root='../../test_altered-hard')

model = Siamese_Network()
# model.load_state_dict(torch.load('Siamese_network_Date:2023_08_02_13_07_31.txt__Epoch:20__MEDIUM+EASY.pt'))
# model.load_state_dict(torch.load('Siamese_network_Date:2023_08_01_03_24_35.txt__Epoch:5.pt'))
model.load_state_dict(torch.load('Siamese_network_Date:2023_08_02_16_45_04.txt__Epoch:100__HARD+MEDIUM+EASY.pt'))



transform_test = transforms.Compose([transforms.Resize((100,100)),
                                    transforms.ToTensor()
                                ])
    
siamese_dataset = SiameseNetworkDataset(directory_test_images, directory_test_altered_images, transform=transform_test)
test_dataloader = DataLoader(siamese_dataset, batch_size=1, shuffle=True)

# Grab one image that we are going to test
dataiter = iter(test_dataloader)
x0, x1, label = next(dataiter)

first = 0
success = 0
for i in range(1000):
    # Iterate over 5 images and test them with the first image (x0)
    if first>0:
        x0, x1, label = next(dataiter)
    first+=1
    # Concatenate the two images together
    concatenated = torch.cat((x0, x1), 0)
    # print(concatenated.size())
    # print("label:", label)
    output1, output2 = model(x0, x1)
    # print("Outputs: ", output1, output2)
    euclidean_distance = F.pairwise_distance(output1, output2)[0]
    print("euclidean dist:", euclidean_distance)
    imshow(torchvision.utils.make_grid(concatenated), f'Dissimilarity: {euclidean_distance.item():.2f}')
    if (euclidean_distance.item() < 0.8 and label==0) or (euclidean_distance.item() >= 0.8 and label==1):
        success+=1
    else:
        imshow(torchvision.utils.make_grid(concatenated), f'Dissimilarity: {euclidean_distance.item():.2f}')


print("Accuracy: ", success/1000)

Accuracy:  0.971
